In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from shapely.geometry import Point
from datetime import datetime, timedelta
import geopandas as gpd
import pandas as pd
import os
import time

# --- Innstillinger ---
client_id = "b24b755b-d187-448b-8676-9fdbb8e6fd80"
client_secret = "ed712d7e-32a2-4afd-8896-b987326ce4f6"
start_date = datetime(2015, 7, 21).date()
end_date = datetime.now().date()
output_dir = "lyn_per_dag"
os.makedirs(output_dir, exist_ok=True)
force_overskriv = True  # <-- NYTT: Tillat overskriving

# Polygon som dekker hele Norge grovt
norge_polygon = "POLYGON((4.5 57.9, 4.5 71.5, 31.5 71.5, 31.5 57.9, 4.5 57.9))"

# Les kommunegrenser
komm = gpd.read_file("https://raw.githubusercontent.com/kjhealy/norway/master/geojson/kommuner.geojson").to_crs("EPSG:4326")

# Les nasjonalgrense (brukes som ekstra filter)
try:
    norge_grense = gpd.read_file("ne_110m_admin_0_countries.shp").to_crs("EPSG:4326")
    norge_grense = norge_grense[norge_grense["NAME"] == "Norway"]
    norge_union = norge_grense.geometry.union_all()
except Exception as e:
    norge_union = None
    print(f"Kunne ikke laste Norge-grense: {e}")

# --- Funksjon for nedlasting og lagring ---
def hent_lyn_for_dato(dato):
    neste = dato + timedelta(days=1)
    outfile = os.path.join(output_dir, f"{dato}.csv")
    if os.path.exists(outfile) and not force_overskriv:
        print(f"{dato} allerede lastet ned.")
        return

    params = {
        "referencetime": f"{dato.isoformat()}T00:00:00Z/{neste.isoformat()}T00:00:00Z",
        "geometry": norge_polygon
    }

    url = "https://frost.met.no/lightning/v0.ualf"
    response = requests.get(url, params=params, auth=HTTPBasicAuth(client_id, client_secret))

    if response.status_code != 200 or not response.text.strip():
        print(f"{dato}: Ingen data ({response.status_code})")
        return

    rows = []
    for line in response.text.strip().splitlines():
        parts = line.split()
        if len(parts) < 10:
            continue
        ts = f"{parts[1]}-{parts[2]}-{parts[3]} {parts[4]}:{parts[5]}:{parts[6]}"
        lat = float(parts[8])
        lon = float(parts[9])
        rows.append({"dato": dato, "timestamp": ts, "lat": lat, "lon": lon})

    df = pd.DataFrame(rows)
    if df.empty:
        return

    gdf = gpd.GeoDataFrame(df, geometry=[Point(xy) for xy in zip(df.lon, df.lat)], crs="EPSG:4326")

    # Filtrer til kun punkter innenfor Norges offisielle grenser
    if norge_union is not None:
        gdf = gdf[gdf.within(norge_union)]

    # Spatial join med kommune
    gdf = gpd.sjoin(gdf, komm, how="left", predicate="within")

    gdf = gdf[["dato", "timestamp", "lat", "lon", "navn"]].rename(columns={"navn": "kommune"})
    gdf.to_csv(outfile, index=False)
    print(f"{dato}: Lagret {len(gdf)} treff.")

# --- Kjør dag for dag ---
dato = start_date
while dato <= end_date:
    try:
        hent_lyn_for_dato(dato)
        time.sleep(1.2)  # for å unngå å overbelaste APIet
    except Exception as e:
        print(f"Feil på {dato}: {e}")
    dato += timedelta(days=1)


2005-07-21: Lagret 139 treff.
2005-07-22: Lagret 574 treff.
2005-07-23: Lagret 0 treff.
2005-07-24: Lagret 37 treff.
2005-07-25: Lagret 84 treff.
2005-07-26: Lagret 41 treff.
2005-07-27: Lagret 460 treff.
2005-07-28: Lagret 700 treff.
2005-07-29: Lagret 9 treff.
2005-07-30: Lagret 6 treff.
2005-07-31: Lagret 2 treff.
2005-08-01: Lagret 47 treff.
2005-08-02: Lagret 0 treff.
2005-08-03: Lagret 223 treff.
2005-08-04: Lagret 1149 treff.
2005-08-05: Lagret 1091 treff.
2005-08-06: Lagret 232 treff.
2005-08-07: Lagret 36 treff.
2005-08-08: Lagret 255 treff.
2005-08-09: Lagret 16 treff.
2005-08-10: Lagret 177 treff.
2005-08-11: Lagret 285 treff.
2005-08-12: Lagret 9 treff.
2005-08-13: Lagret 0 treff.
2005-08-14: Lagret 1 treff.
2005-08-15: Lagret 0 treff.
2005-08-16: Lagret 0 treff.
2005-08-17: Lagret 0 treff.
2005-08-18: Lagret 0 treff.
2005-08-19: Lagret 16 treff.
2005-08-20: Lagret 4 treff.
2005-08-21: Lagret 0 treff.
2005-08-22: Lagret 7 treff.
2005-08-23: Lagret 61 treff.
2005-08-24: Lagr

In [4]:
import pandas as pd
import geopandas as gpd
import os

# --- Les og kombiner alle daglige filer ---
alle_filer = [f"lyn_per_dag/{fil}" for fil in os.listdir("lyn_per_dag") if fil.endswith(".csv")]
df = pd.concat((pd.read_csv(f) for f in alle_filer), ignore_index=True)

# --- Tell antall per kommune ---
per_kommune = df["kommune"].value_counts().reset_index()
per_kommune.columns = ["kommune", "lyn_treff"]

# --- Legg til total og frekvens ---
totalt = per_kommune["lyn_treff"].sum()
per_kommune["andel (%)"] = per_kommune["lyn_treff"] / totalt * 100
per_kommune["lyn_per_år"] = per_kommune["lyn_treff"] / 20  # 20 år

# --- Les kommunegrenser og beregn areal ---
komm_gdf = gpd.read_file("https://raw.githubusercontent.com/kjhealy/norway/master/geojson/kommuner.geojson").to_crs(epsg=32633)
komm_gdf["areal_km2"] = komm_gdf.geometry.area / 1e6  # i km²
komm_gdf = komm_gdf[["navn", "areal_km2"]].rename(columns={"navn": "kommune"})

# --- Slå sammen for å få areal ---
per_kommune = pd.merge(per_kommune, komm_gdf, on="kommune", how="left")

# --- Beregn lyn per kvadratkilometer ---
per_kommune["lyn_per_km2"] = per_kommune["lyn_treff"] / per_kommune["areal_km2"]

# --- Lagre som CSV ---
per_kommune.to_csv("lynstatistikk_2005_2025_med_areal.csv", index=False)

# --- Skriv også totalen til skjerm ---
print(f"Totalt antall lynnedslag: {totalt}")


C:\Users\jakob\AppData\Local\Temp\ipykernel_12972\2406232016.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat((pd.read_csv(f) for f in alle_filer), ignore_index=True)


Totalt antall lynnedslag: 2431944


In [3]:
import requests
from requests.auth import HTTPBasicAuth
from shapely.geometry import Point
from datetime import datetime, timedelta
import geopandas as gpd
import pandas as pd
import os
import time

# --- Innstillinger ---
client_id = "b24b755b-d187-448b-8676-9fdbb8e6fd80"
client_secret = "ed712d7e-32a2-4afd-8896-b987326ce4f6"
start_date = datetime(2005, 7, 21).date()
end_date = datetime.now().date()
output_dir = "lyn_per_dag"
os.makedirs(output_dir, exist_ok=True)
force_overskriv = True  # <-- NYTT: Tillat overskriving

# Polygon som dekker hele Norge grovt
norge_polygon = "POLYGON((4.5 57.9, 4.5 71.5, 31.5 71.5, 31.5 57.9, 4.5 57.9))"

# Les kommunegrenser
komm = gpd.read_file("https://raw.githubusercontent.com/kjhealy/norway/master/geojson/kommuner.geojson").to_crs("EPSG:4326")

komm["areal_km2"] = komm.to_crs(epsg=32633).geometry.area / 10**6
komm[["navn", "areal_km2"]].sort_values("areal_km2", ascending=False).head()

,navn,areal_km2
240,Guovdageaidnu  Kautokeino,9727.122531
238,Porsanger  Porsāngu  Porsanki,6012.740711
249,Karlsøy,5887.708244
186,Lebesby,5751.385338
239,Kárásjohka  Karasjok,5469.599705
